#### Creating the Grid
<br>
We start by defining a function create_grid that creates a 2D grid to represent the environment.
This function takes two arguments: width for the grid's width in cells and height for its height in cells. The grid is initialized with all cells set to 0, representing free space.

In [1]:
def create_grid(width, height):
    grid = []
    for _ in range(height):
        row = [0] * width
        grid.append(row)
    return grid


<br>

#### Setting Obstacles
<br>
The set_obstacle function is used to mark specific cells in the grid as obstacles, with the value 1. It takes three arguments: grid (the grid map representation), x (the X-coordinate of the cell), and y (the Y-coordinate of the cell). Obstacles are set by changing the value of the specified cell in the grid.

In [2]:
def set_obstacle(grid, x, y):
    if 0 <= x < len(grid[0]) and 0 <= y < len(grid):
        grid[y][x] = 1

<br>

#### Printing the Grid
<br>To visualize the grid, we use the print_grid function, which prints the grid map.

In [3]:
def print_grid(grid):
    for row in grid:
        print(" ".join([str(cell) for cell in row]))

<br>

#### Example Usage
<br>



In [4]:
width = 10
height = 10

grid = create_grid(width, height)

set_obstacle(grid, 2, 2)
set_obstacle(grid, 5, 3)
set_obstacle(grid, 1, 8)


print_grid(grid)


0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0


<br><br><br>

#### Cost Function
The cost function cost_function assigns a cost value to each action the robot takes. It considers factors such as terrain type (carpet vs. hardwood) and energy expenditure for turns to determine the cost of movement.

In [5]:
def cost_function(action, terrain_type):
    # Default unit cost for movement
    base_cost = 1

    if terrain_type == "carpet":
        # Higher cost for moving on carpet
        movement_cost = base_cost * 2
    else:
        movement_cost = base_cost

    if action in ["LEFT", "RIGHT"]:
        # Add turn cost to movement cost for turning actions
        total_cost = movement_cost + 2
    else:
        # No extra cost for straight movements
        total_cost = movement_cost

    return total_cost


#### Cost function Test
<br>
The cost_function calculates the cost of a robot's action based on the terrain type and the action taken. It considers two factors, movement cost and turn cost. The movement cost is doubled if the robot is on carpet terrain, and the turn cost is higher for turning actions (LEFT or RIGHT) compared to straight movements (UP or DOWN).

Testing Expectations:

1. When the robot moves UP or DOWN on hardwood terrain, the cost should be 1.
2. When the robot moves LEFT or RIGHT on hardwood terrain, the cost should be 3.
3. When the robot moves UP or DOWN on carpet terrain, the cost should be 2.
4. When the robot moves LEFT or RIGHT on carpet terrain, the cost should be 4.




When the robot moves UP or DOWN on hardwood terrain, the cost should be 1:


In [6]:
print(cost_function("UP", "hardwood"))  # Output: 1
print(cost_function("DOWN", "hardwood"))  # Output: 1

1
1


When the robot moves LEFT or RIGHT on hardwood terrain, the cost should be 3:

In [7]:
print(cost_function("LEFT", "hardwood"))  # Output: 3
print(cost_function("RIGHT", "hardwood"))  # Output: 3

3
3


When the robot moves UP or DOWN on carpet terrain, the cost should be 2 (doubled movement cost):

In [8]:
print(cost_function("UP", "carpet"))  # Output: 2
print(cost_function("DOWN", "carpet"))  # Output: 2

2
2


When the robot moves LEFT or RIGHT on carpet terrain, the cost should be 4:

In [9]:
print(cost_function("LEFT", "carpet"))  # Output: 4
print(cost_function("RIGHT", "carpet"))  # Output: 4

4
4


<br>

#### Heuristic Function
The heuristic function estimates the remaining distance for the robot to reach the cleaning target. It considers obstacles in the environment and penalizes paths that require extensive navigation around them.

<br>

#### Calculate Manhattan Distance Function

Calculating the Manhattan distance between two points, representing a simple heuristic that estimates the remaining distance for the robot to reach the cleaning target. This function ignores obstacles in the environment.

In [10]:
def manhattan_distance(start, end):
    return abs(start[0] - end[0]) + abs(start[1] - end[1])


#### Handle Obstacle Function

The purpose of this function is to handle obstacles in the environment when calculating the heuristic. It checks if a specific grid cell contains an obstacle and penalizes paths that would require the robot to navigate around obstacles. The function calculates the distances between the robot's current position, the obstacle, and the target position to determine if the path should be penalized.

<br>

The way this is working is basically helping the robot select paths that are not only direct (shortest, straight without detours) but also avoid obstacles between the robot's current position, the obstacle, and the target position.


In [11]:
def handle_obstacle(current_position, target_position, grid, heuristic_value, row, col):
    if grid[row][col] == 1:
        obstacle_position = (col, row)
        obstacle_distance = manhattan_distance(current_position, obstacle_position)
        target_obstacle_distance = manhattan_distance(obstacle_position, target_position)

        # Penalize paths that require navigation around obstacles
        print(f"distance to obstacles: {obstacle_distance}, distance from obstacle to target: {target_obstacle_distance}, heuristic value: {heuristic_value}")
        if obstacle_distance + target_obstacle_distance < heuristic_value:

            # Penalize heavily for navigating around obstacles
            heuristic_value += 10
    return heuristic_value

#### Testing handle_obstacle

This function is expected to handle obstacles encountered during path finding and update the heuristic value accordingly.

The higher heuristic value indicates that encountering an obstacle has made the path longer and more complex, leading to a higher estimated cost to reach the goal. This tells us that the algorithm is appropriately considering obstacles and adjusting its estimates accordingly for more accurate path finding.

In [12]:
current_position = (0, 0)
target_position = (9, 9)

print_grid(grid)

heuristic_value = 20

# Check a cell with an obstacle and update the heuristic value
updated_heuristic_value = handle_obstacle(current_position, target_position, grid, heuristic_value, 2, 2)
print(f"Updated heuristic value: {updated_heuristic_value}")

0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
distance to obstacles: 4, distance from obstacle to target: 14, heuristic value: 20
Updated heuristic value: 30


#### Heuristic Function

This code is the heuristic function for path finding, taking into account obstacles in the grid. It calculates the Manhattan distance between the current position and the target position as the initial heuristic value. Then, it iterates through each cell in the grid to check for obstacles, updating the heuristic value based on the obstacle penalty using the `handle_obstacle` function. Finally, it returns the updated heuristic value, incorporating penalties for navigating around obstacles to provide a more accurate estimate of the remaining distance to reach the target.

In [13]:
def heuristic(current_position, target_position, grid):

    # Calculate the Manhattan distance
    heuristic_value = manhattan_distance(current_position, target_position)

    # Penalize paths that require navigation around obstacles
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            heuristic_value = handle_obstacle(current_position, target_position, grid, heuristic_value, row, col)

    return heuristic_value


<br>

#### Test Heuristic Function

This test scenario calculates the estimated remaining distance for a robot to reach a target position from a given current position on a grid potentially containing obstacles. The test sets the current position as `(2, 2)` and the target position as `(9, 9)`. It then calls the `heuristic` function with these positions and the grid, aiming to verify the function's functionality in calculating the heuristic value considering Manhattan distance and obstacle penalties. The resulting `estimated_distance` value from the test is printed to demonstrate the estimated remaining distance, taking into account obstacles in the environment.

In [14]:
current_position = (2, 2)
target_position = (9, 9)

estimated_distance = heuristic(current_position, target_position, grid)
print(f"Estimated remaining distance with obstacles: {estimated_distance}")


distance to obstacles: 0, distance from obstacle to target: 14, heuristic value: 14
distance to obstacles: 4, distance from obstacle to target: 10, heuristic value: 14
distance to obstacles: 7, distance from obstacle to target: 9, heuristic value: 14
Estimated remaining distance with obstacles: 14
